# ibm_db.execute_many()

## Purpose:

Execute an SQL statement that has been prepared by the __ibm_db.prepare()__ API, using the parameter sequences or mappings specified.

## Syntax:

`mixed ibm_db.execute_many( IBM_DBStatement `*`preparedStmt,`*` tuple `*`seqOfParamValues`*` )`

## Parameters:

* __*preparedStmt*__ &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp; &nbsp; : A prepared SQL statement returned from the __ibm_db.prepare()__ API.
* __*seqOfParamValues*__&nbsp; &nbsp; : A tuple of tuples that contain values for each parameter marker used in the prepared SQL statement (that was speficied in the __*preparedStmt*__ parameter). 

## Return values:

* If __successful__, an integer representing the number of rows that were inserted, updated, or deleted by the SQL statement specified in the __*preparedStmt*__ parameter.
* If __unsuccessful__, the value `False`.

## Description:

The __ibm_db.execute_many()__ API is used to execute an SQL statement that has been prepared by the __ibm_db.prepare()__ API, using a set of parameter sequences or mappings. Essentially, this API is used to perform bulk __INSERT__, __UPDATE__, and __DELETE__ operations. (To achieve optimum performance, this API utilizes *array input chaining* to minimize the number of round trips that have to be made between the client and the server.)

## Example:

In [1]:
#----------------------------------------------------------------------------------------------#
#  NAME:     ibm_db-execute_many.py                                                            #
#                                                                                              #
#  PURPOSE:  This program is designed to illustrate how to use the ibm_db.execute_many() API.  #
#                                                                                              #
#            Additional APIs used:                                                             #
#                 ibm_db.prepare()                                                             #
#                 ibm_db.fetch_tuple()                                                         #
#                                                                                              #
#----------------------------------------------------------------------------------------------#
#                     DISCLAIMER OF WARRANTIES AND LIMITATION OF LIABILITY                     #
#                                                                                              #
#  (C) COPYRIGHT International Business Machines Corp. 2018, 2019 All Rights Reserved          #
#  Licensed Materials - Property of IBM                                                        #
#                                                                                              #
#  US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA    #
#  ADP Schedule Contract with IBM Corp.                                                        #
#                                                                                              #
#  The following source code ("Sample") is owned by International Business Machines            #
#  Corporation ("IBM") or one of its subsidiaries and is copyrighted and licensed, not sold.   #
#  You may use, copy, modify, and distribute the Sample in any form without payment to IBM,    #
#  for the purpose of assisting you in the creation of Python applications using the ibm_db    #
#  library.                                                                                    #
#                                                                                              #
#  The Sample code is provided to you on an "AS IS" basis, without warranty of any kind. IBM   #
#  HEREBY EXPRESSLY DISCLAIMS ALL WARRANTIES, EITHER EXPRESS OR IMPLIED, INCLUDING, BUT NOT    #
#  LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE. #
#  Some jurisdictions do not allow for the exclusion or limitation of implied warranties, so   #
#  the above limitations or exclusions may not apply to you. IBM shall not be liable for any   #
#  damages you suffer as a result of using, copying, modifying or distributing the Sample,     #
#  even if IBM has been advised of the possibility of such damages.                            #
#----------------------------------------------------------------------------------------------#

# Load The Appropriate Python Modules
import sys         # Provides Information About Python Interpreter Constants And Functions
import ibm_db      # Contains The APIs Needed To Work With Db2 Databases

#----------------------------------------------------------------------------------------------#
# Import The Db2ConnectionMgr Class Definition, Attributes, And Methods That Have Been Defined #
# In The File Named "ibm_db_tools.py"; This Class Contains The Programming Logic Needed To     #
# Establish And Terminate A Connection To A Db2 Server Or Database                             #
#----------------------------------------------------------------------------------------------#
from ibm_db_tools import Db2ConnectionMgr

#----------------------------------------------------------------------------------------------#
# Import The get_row_count() Function That Has Been Defined In The File Named                  #
# "ibm_db_tools.py"; This Function Contains The Programming Logic Needed To Obtain And         #
# Display The Number Of Rows (Records) Found In A Db2 Database Table.                          #
#----------------------------------------------------------------------------------------------#
from ibm_db_tools import get_row_count

#----------------------------------------------------------------------------------------------#
# Import The ipynb_exit Class Definition, Attributes, And Methods That Have Been Defined In    #
# The File Named "ipynb_exit.py"; This Class Contains The Programming Logic Needed To Allow    #
# "exit()" Functionality To Work Without Raising An Error Or Stopping The Kernel If The        #
# Application Is Invoked In A Jupyter Notebook                                                 #
#----------------------------------------------------------------------------------------------#
from ipynb_exit import exit

# Define And Initialize The Appropriate Variables
dbName = "SAMPLE"
userID = "db2inst1"
passWord = "Passw0rd"
dbConnection = None
returnCode = None
preparedStmt = False

# Create An Instance Of The Db2ConnectionMgr Class And Use It To Connect To A Db2 Database
conn = Db2ConnectionMgr('DB', dbName, '', '', userID, passWord)
conn.openConnection()
if conn.returnCode is True:
    dbConnection = conn.connectionID
else:
    conn.closeConnection()
    exit(-1)

# Display The Number Of Rows That Currently Exist In The DEPARTMENT Table
returnCode = get_row_count(dbConnection, 'DEPARTMENT')
if returnCode is False:
    conn.closeConnection()
    exit(-1)

# Define The INSERT Statement (With Parameter Markers) That Is To Be Used To Add Data
# To The DEPARTMENT Table
sqlStatement = "INSERT INTO department VALUES(?, ?, ?, 'E01', NULL)"

# Prepare The SQL Statement Just Defined
print("Preparing the SQL statement \"" + sqlStatement + "\" ... ", end="")
try:
    preparedStmt = ibm_db.prepare(dbConnection, sqlStatement)
except Exception:
    pass

# If The SQL Statement Could Not Be Prepared By Db2, Display An Error Message And Exit
if preparedStmt is False:
    print("\nERROR: Unable to prepare the SQL statement specified.\n")
    conn.closeConnection()
    exit(-1)

# Otherwise, Complete The Status Message
else:
    print("Done!\n")

# Create A List Of Data Values That Are To Be Supplied For The Parameter Markers Coded
# In The INSERT Statement Specified
pmValues = (('K22', 'SALES', '000110'), 
            ('L22', 'FINANCE', '000120'), 
            ('M22', 'HUMAN RESOURCES', '000130'))

# Execute The SQL Statement Just Prepared
print("Executing the prepared SQL statement ... ", end="")
try:
    returnCode = ibm_db.execute_many(preparedStmt, pmValues)
except Exception:
    pass

# If The SQL Statement Could Not Be Executed, Display An Error Message And Exit 
if returnCode is True:
    print("\nERROR: Unable to execute the SQL statement specified.\n")
    conn.closeConnection()
    exit(-1)

# Otherwise, Complete The Status Message And Display The Number Of Records Added
else:
    print("Done!\n")
    print("Number of records added : {}\n" .format(returnCode))

    # Display The Number Of Rows That Exist In The DEPARTMENT Table Now
    # (The Number Returned Should Reflect That Three Rows Were Added)
    returnCode = get_row_count(dbConnection, 'DEPARTMENT')

    # Close The Database Connection That Was Opened Earlier
    conn.closeConnection()

# Return Control To The Operating System
exit()


Connecting to the SAMPLE database ... Done!

Number of records found in the DEPARTMENT table: 14

Preparing the SQL statement "INSERT INTO department VALUES(?, ?, ?, 'E01', NULL)" ... Done!

Executing the prepared SQL statement ... Done!

Number of records added : 3

Number of records found in the DEPARTMENT table: 17

Disconnecting from the SAMPLE database ... Done!

